# Extracting (1C) post cycling capacity of cells

Author: Roger Ho

this notebook aims to extract the 1C capacity of cells after accelerated cycle life testing. the function used here is an excerpt from `eol.py`, a class that encapsulates end-of-life cell data/characteristics

caveats:
- c/20 capacity diagnostic tests are ran at set intervals
- c/20 capacity tests take much longer to run, meaning high chance final cycle is c/20 test

import standard libraries and `FormationCell` from `formation.py` for data encapsulation

In [5]:
# import FormationCell from formation
import numpy as np
import pandas as pd
import math

# configure paths
import sys
import os

if os.path.basename(os.getcwd()) == 'rho':
    print('changing path...')
    os.chdir('../../../')
    sys.path.insert(0, 'src/')
    
from formation import FormationCell

define function for extracting post cycling capacities. basic logic is:
1. get aging cycle data
2. grab last N (currently N = 8) capacities from the last N cycles
3. if coefficient of variation is too high (> 0.1), then

+ <ol type="a">
    <li>calculate interquartile range (IQR)</li>
    <li>estimate upper bounds using IQR</li>
    <li>prune anomalies using upper bound</li>
</ol>

first define function to prune data with IQR

In [6]:
def prune_anomalies(list_in):
    q75,q25 = np.percentile(list_in,[75,25])
    intr_qr = q75-q25
    
    qmax = q75+(1.5*intr_qr)
    qmin = q25-(1.5*intr_qr)
    
    # prune using IQE
    pruned_list = [cap for cap in list_in if (cap < qmax and cap > qmin)]
    
    return pruned_list

In [7]:
def get_post_cycling_capacity(cellid):
    # init formation cell
    formation_cell = FormationCell(cellid)
    df = formation_cell.get_aging_data_cycles() # get formation data

    # plotting for verification
    # cycle_numbers = np.unique(df['Cycle Number'])
    
    # for cycle in cycle_numbers:
    #     cycle_df = df[df['Cycle Number'] == cycle]
    #     print(cycle_df['Discharge Capacity (Ah)'])
        
    #     fig, ax = plt.subplots()
    #     ax.plot(cycle_df['Cycle Net Capacity (Ah)'])
    #     plt.show()
        
    # index for cycle containing the final discharge capacity
    CYCLE_INDEX_LAST = np.max(df['Cycle Number'])

    # save last N measured capacities to list
    N = 12 # 12 is min number to prune away 3 anomalies (max that i've seen)
    last_N_capacities = list()

    # get capacities of last 5 cycles
    for i in range(CYCLE_INDEX_LAST - N + 1, CYCLE_INDEX_LAST + 1):
        cycle_discharge_capacity = np.max(df[df['Cycle Number'] == i]['Discharge Capacity (Ah)'])
        # only append if not nan (empty dfs give nan)
        if not math.isnan(cycle_discharge_capacity):
            last_N_capacities.append(cycle_discharge_capacity)

    # check for c/20 tests using coefficient of variation
    CV = np.std(last_N_capacities) / np.mean(last_N_capacities)
    if CV > 0.1: # significant
        # calculate IQR
        q75,q25 = np.percentile(last_N_capacities,[75,25])
        intr_qr = q75-q25
        
        qmax = q75+(1.5*intr_qr)
        qmin = q25-(1.5*intr_qr)
        
        # prune using IQE
        last_N_capacities = prune_anomalies(last_N_capacities)
        
        # print num of pruned entries
        # print(f'{N - len(last_N_capacities)} entries pruned when getting post cycling capacity')
        
        # recalculating COV
        CV = np.std(last_N_capacities) / np.mean(last_N_capacities)
    
    if CV > 0.1: # prune till ok
        print(f'\nWARNING: high coefficient of variation of ', CV, ' pruning again...')
        last_N_capacities = prune_anomalies(last_N_capacities)
        CV = np.std(last_N_capacities) / np.mean(last_N_capacities)
        print(f'last {N} capacities, pruned: {last_N_capacities}')
        q75,q25 = np.percentile(last_N_capacities,[75,25])
        intr_qr = q75-q25
        print("cutoff was ", q75+(1.5*intr_qr),'\n')
    
    # final capacity is now last of the list
    final_discharge_capacity = last_N_capacities[-1]

    # return
    return final_discharge_capacity

run for all 40 cells and save to dataframe

In [8]:
for i in range(1, 41):
    print("cell",i,": ", get_post_cycling_capacity(i))

cell 1 :  0.2314204852
cell 2 :  0.0642911201

last 12 capacities, pruned: [0.7113994469, 0.6722224121, 0.6419788624, 0.6106689008, 0.5857907865, 0.5690297784, 0.5526177191, 0.5415908429, 0.5300898549, 0.5181741898, 0.5138489764, 0.5107416296]
cutoff was  0.7555745700625 

cell 3 :  0.5107416296
cell 4 :  0.2933887219
cell 5 :  0.1798601926
cell 6 :  0.0221319587
cell 7 :  0.0953049287
cell 8 :  0.1521974505
cell 9 :  0.0632311733
cell 10 :  0.035035103
cell 11 :  1.1378696223
cell 12 :  1.040720425
cell 13 :  0.858923615
cell 14 :  0.8852171388
cell 15 :  0.762523983799999
cell 16 :  0.827490672500002
cell 17 :  0.4921636331
cell 18 :  0.7377981774
cell 19 :  0.9007932441
cell 20 :  0.4747772034
cell 21 :  1.2186329858
cell 22 :  0.4240728364
cell 23 :  1.0329127909
cell 24 :  0.9867838941
cell 25 :  0.9628010554
cell 26 :  1.1036038029
cell 27 :  1.1740315535
cell 28 :  0.8437031887

last 12 capacities, pruned: [0.5783187567, 0.5697737562, 0.5663832842, 0.5387862622, 0.5120056692, 0.